## Day 7: No Space Left On Device

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day07/puzzle.ipynb)



### Parsing

In [1]:
#!value --name sampleRaw
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k

Formally, the given input is not a list of commands. It is commands plus their output when applicable. But in order to simplify the parsing process let us consider the lines of the `ls` command output to be separate commands, as they are distinct in format.

In [2]:
type Command = 
    | CDRoot
    | CDUp
    | CD of string
    | Ls
    | Dir of string
    | File of int64*string

In [3]:
#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder
open Farkle.Builder.Regex

#load "../common/common.fsx"

Installed Packages Farkle, 6.3.2

In [4]:
let private name = 
    [
        chars PredefinedSets.AllLetters
        char '.'
    ] |> choice
    |> atLeast 1
    |> terminal "Name" (T(fun _ x -> x.ToString()))
let private number = Terminals.int64 "Number"
let private command = "Command" ||= [
    !& "$ cd /" =% CDRoot
    !& "$ cd .." =% CDUp
    !& "$ ls" =% Ls
    !& "$ cd" .>>. name => fun dirName -> CD dirName
    !& "dir" .>>. name => fun dirName -> Dir dirName
    !@ number .>>. name => fun size fileName -> File (size,fileName)
]
let private parser = RuntimeFarkle.build command

let private nameParser = RuntimeFarkle.build name

let parseCommand s = 
    s
    |> RuntimeFarkle.parseString parser
    |> Result.get


In [5]:
#!share sampleRaw --from value

let sampleCommands = sampleRaw |> Pattern1.read parseCommand
sampleCommands

index type value 0 FSI_0005+Command CDRoot 1 FSI_0005+Command Ls 2 FSI_0005+Command+Dir Dir "a" Item a 3 FSI_0005+Command+File File (14848514L, "b.txt") Item1 14848514 Item2 b.txt 4 FSI_0005+Command+File File (8504156L, "c.dat") Item1 8504156 Item2 c.dat 5 FSI_0005+Command+Dir Dir "d" Item d 6 FSI_0005+Command+CD CD "a" Item a 7 FSI_0005+Command Ls 8 FSI_0005+Command+Dir Dir "e" Item e 9 FSI_0005+Command+File File (29116L, "f") Item1 29116 Item2 f 10 FSI_0005+Command+File File (2557L, "g") Item1 2557 Item2 g 11 FSI_0005+Command+File File (62596L, "h.lst") Item1 62596 Item2 h.lst 12 FSI_0005+Command+CD CD "e" Item e 13 FSI_0005+Command Ls 14 FSI_0005+Command+File File (584L, "i") Item1 584 Item2 i 15 FSI_0005+Command CDUp 16 FSI_0005+Command CDUp 17 FSI_0005+Command+CD CD "d" Item d 18 FSI_0005+Command Ls 19 FSI_0005+Command+File File (4060174L, "j") Item1 4060174 Item2 j (3 more)

In [6]:
#!value --name actualRaw --from-file ./data_actual.txt

In [7]:
#!share actualRaw --from value
let actualCommands = actualRaw |> Pattern1.read parseCommand

### Part 1

We always start from the root of the filesystem, which is rather helpful. The filesystem can be described in the immutable recursive type called `Dir`. 

In [8]:
type File = { Name: string; Size: int64 }
type Dir =
    { Name: string
      Nested: Map<string, Dir>
      Files: Map<string, File>
      }
    with member this.TotalSize = 
            (this.Nested |> Map.values |> Seq.sumBy (fun x -> x.TotalSize))
            + 
            (this.Files |> Map.values |> Seq.sumBy (fun x -> x.Size))

Formatter.SetPreferredMimeTypesFor(typedefof<Dir>, "text/plain")
let rec private formatDir (sb : StringBuilder) (dir : Dir) depth = 
    sb.Append(String.replicate (depth * 2) " ")
      .Append($"- {dir.Name} (dir, totalsize={dir.TotalSize})")
      .AppendLine()
      |> ignore
    dir.Nested
    |> Seq.sortBy (fun nested -> nested.Key)
    |> Seq.iter (fun nested -> formatDir sb nested.Value (depth+1))
    dir.Files
    |> Seq.sortBy (fun file -> file.Key)
    |> Seq.iter (fun file -> 
        sb.Append(String.replicate (depth * 2 + 2) " ")
          .AppendLine($"- {file.Key} (file, size={file.Value.Size})")
        |> ignore
    )
Formatter.Register<Dir>((fun x -> 
    let sb = StringBuilder()
    formatDir sb x 0
    sb.ToString()
), "text/plain")

To update nested records as we read the input, we can make use of `lenses`. They allow to focus on a specific part of a data structure for reading and updating, and their composable nature allows to work with nested structures. As we read the input, we are going to maintain the lens focusing on the current directory composed of simpler lenses.

In [9]:
#r "nuget: FSharpPlus, 1.5.0"
open FSharpPlus.Lens
open FSharpPlus.Data
module Dir = 
    let empty name = 
        { Name = name; Nested = Map.empty; Files = Map.empty;  }
    let inline _files f dir = 
        f dir.Files <&> (fun files -> { dir with Files = files})
    let inline _nested f dir = 
        f dir.Nested <&> (fun nested -> { dir with Nested = nested;})
    let inline _filesItem name file = 
        _files << Map._item name <| file
    let inline _nestedItem name nested = 
        _nested << Map._item name <| nested

Installed Packages FSharpPlus, 1.5.0

In the example below we update the tree by adding a nested dir called `child`

In [10]:
let private example = Dir.empty "root"
let private child = Dir.empty "child"
setl (Dir._nestedItem "child") (Some child) example

- root (dir, totalsize=0)
  - child (dir, totalsize=0)


In the example below Dir `child2` is not added as there is no intermediate `child1` node

In [11]:
let private example = Dir.empty "root"
let private child2 = Dir.empty "child2"
setl (Dir._nestedItem "child1" << _Some << Dir._nestedItem "child2") 
    (Some child2) example

- root (dir, totalsize=0)


In the example below we fix the issue by adding the `child1` node first and calling the composed lens against the newly created root.

In [12]:
let private example = Dir.empty "root"
let private child1 = Dir.empty "child1"
let private child2 = Dir.empty "child2"

setl (Dir._nestedItem "child1") (Some child1) example
|> setl (Dir._nestedItem "child1" << _Some << Dir._nestedItem "child2") (Some child2)

- root (dir, totalsize=0)
  - child1 (dir, totalsize=0)
    - child2 (dir, totalsize=0)


So, to solve the problem we can do the following:
1. For each `cd x` command a new `Dir` will be added to the filesystem representation using the current lens, and the current lens will be changed to focus on the new `Dir`.
2. Every discovered file size will be added to the filesystem representation using the current lens, and `TotalSize`-s will be recalculated automatically
3. Each `cd ..` will roll back to the previous lens.

In [13]:
type private Lens = { Current: (Dir option -> Identity<Dir option>) -> Dir -> Identity<Dir>; Parent: Lens option }
    with static member Empty = { Current = (fun f x -> f (Some x) <&> (Option.get)); Parent = None }

let private folder (fs: Dir, lens: Lens) (command : Command) = 
    match command with
        | CDRoot -> (fs, lens)
        | Ls -> (fs, lens)
        | Dir _ -> (fs, lens)
        | CDUp -> (fs, lens.Parent |> Option.get)
        | CD dirName -> 
            let newLens = 
                { Current = lens.Current << _Some << Dir._nestedItem dirName; Parent = Some lens }
            let newDir = Dir.empty dirName
            let newFs = setl newLens.Current (Some newDir) fs
            (newFs, newLens)
        | File (fileSize, fileName) -> 
            let newFile = {Name = fileName; Size = fileSize}
            let newFs = setl (lens.Current << _Some << Dir._filesItem fileName) (Some newFile) fs
            (newFs, lens)

let execute (commands : seq<Command>) = 
    let root = Dir.empty "/"
    let (fs, _) = commands |> Seq.fold folder (root, Lens.Empty)

    fs


In [14]:
let sampleFs = execute sampleCommands
sampleFs

- / (dir, totalsize=48381165)
  - a (dir, totalsize=94853)
    - e (dir, totalsize=584)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - d (dir, totalsize=24933642)
    - d.ext (file, size=5626152)
    - d.log (file, size=8033020)
    - j (file, size=4060174)
    - k (file, size=7214296)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)


Now we have a decent image of the filesystem as a tree and can do with it whatever is needed to be done.

In [15]:
let rec collectDirs (root: Dir) =
    seq {
        yield root
        yield! root.Nested |> Map.values |> Seq.collect collectDirs
    }

let sumBelow100k (root: Dir) =     
    collectDirs root
    |> Seq.map (fun x -> x.TotalSize)
    |> Seq.filter (fun x -> x <= 100000)
    |> Seq.sum

sampleFs
|> sumBelow100k

95437

For the actual data:

In [16]:
let actualFs = execute actualCommands
actualFs |> sumBelow100k

1307902

### Part 2

In [17]:
let directoryToDelete (root : Dir) = 
    let total = 70000000L
    let required = 30000000L
    let unused = total - root.TotalSize
    let toFree = required - unused
    
    collectDirs root
    |> Seq.filter (fun x -> x.TotalSize >= toFree)
    |> Seq.minBy (fun x -> x.TotalSize)

In [18]:
sampleFs
|> directoryToDelete
|> displayPipe
|> (fun x -> x.TotalSize)

- d (dir, totalsize=24933642)
  - d.ext (file, size=5626152)
  - d.log (file, size=8033020)
  - j (file, size=4060174)
  - k (file, size=7214296)


24933642

For the actual data:

In [19]:
actualFs
|> directoryToDelete
|> (fun x -> x.TotalSize)

7068748